In [4]:
# import twint
import pandas as pd
pd.set_option('display.max_columns', 50)

In [5]:
negative_tweets = None
positive_tweets = None

def load_from_csv(suffix_file_name='tweet_temp.csv'):
    # negative_tweets = pd.DataFrame(columns=df.columns)
    # positive_tweets = pd.DataFrame(columns=df.columns)
    negative_tweets = pd.read_csv('negative_' + suffix_file_name)
    positive_tweets = pd.read_csv('positive_' + suffix_file_name)
    return positive_tweets, negative_tweets

In [6]:
import uuid
import time
from threading import Thread

def dump_labeled_data(path='tweet_temp.csv'):
    global positive_tweets, negative_tweets
    print('Positive: ', len(positive_tweets), ' Negative: ', len(negative_tweets),
          ' Total : ', len(positive_tweets) + len(negative_tweets))
    positive_tweets.to_csv('positive_' + path)
    negative_tweets.to_csv('negative_' + path)
    
def do_label(df):
    global positive_tweets
    global negative_tweets
    print('D for Depressed, H for Healthy, P for pass')
    print('Num of tweets: #{}'.format(df.count()))
    possible_values = ['D', 'H', 'P']
    for idx, row in df.iterrows():
        while True:
            prompt = input('#{}: {} (D/H/P)'.format(idx + 1, row.tweet))
            if prompt in possible_values:
                break
        if prompt == 'H':
            positive_tweets = positive_tweets.append(row)
        elif prompt == 'D':
            negative_tweets = negative_tweets.append(row)
        print()

def do_scrap(query_string, limit = 25):
    scrap_dir = 'tweet-{}'.format(uuid.uuid1())
    c2 = twint.Config()
    c2.Limit = limit
    c2.Search = query_string
    c2.Store_csv = True
    c2.Output = scrap_dir
    try:
        twint.run.Search(c2)
    except:
        pass

    return scrap_dir
    
def do_read_pd(path):
    return pd.read_csv(path + '/tweets.csv')
    

## Example Usage

In [8]:
pos_rani, neg_rani = load_from_csv('tweet_albert.csv')

In [11]:
pos, neg = load_from_csv()

In [12]:
pos_rehan, neg_rehan = load_from_csv('tweet_rehan.csv')

In [19]:
pos_ids = set(pos['id'])
neg_ids = set(neg['id'])
pos_ids

{397730606202617857,
 424554780669640706,
 531345043567546368,
 1117335250797879297,
 1117463608730869765,
 1117717061252202496,
 1118177510590533638,
 1118897219833196549,
 1120346045181915136,
 1120720457777340417,
 1120985277122498565,
 1121079222594723841,
 1121406367326072832,
 1121515244893786112,
 1121961286206545921,
 1122004364103520257,
 1122114706309672961,
 1124109680421163008,
 1124140253193748480,
 1125348717786058752,
 1125405735473385474,
 1126111322184142848,
 1126117061116088321,
 1126354951855165440,
 1126438515812622336,
 1126488475962728448,
 1126512444673646594,
 1126523846599663617,
 1126616157358714880,
 1126670738734194688,
 1126848615287144448,
 1126857941947015168,
 1126894271368314882,
 1127262863112974338,
 1127650851529453568,
 1127933327736881152,
 1127935635400957954,
 1127946401139314688,
 1127951606543077378,
 1127951818669973505,
 1127952450374160384,
 1127956646519435267,
 1127957136640602113,
 1127957723318874112,
 1127958438330290176,
 112796039501

In [23]:
for idx, row in pos_rehan.iterrows():
    if row['id'] not in pos_ids:
        pos = pos.append(row)
        pos_ids.add(row['id'])

In [24]:
for idx, row in neg_rehan.iterrows():
    if row['id'] not in neg_ids:
        neg = neg.append(row)
        neg_ids.add(row['id'])

In [27]:
for idx, row in pos_rani.iterrows():
    if row['id'] not in pos_ids:
        pos = pos.append(row)
        pos_ids.add(row['id'])

In [28]:
for idx, row in neg_rani.iterrows():
    if row['id'] not in neg_ids:
        neg = neg.append(row)
        neg_ids.add(row['id'])

In [31]:
positive_tweets = pos
negative_tweets = neg

In [12]:
def read_pd_dataframe(suffixes = ['rani', 'rehan', 'temp', 'albert', 'progress_iv']):
    positives, negatives = [], []
    for suffix in suffixes:
        positive = pd.read_csv("positive_tweet_" + suffix + ".csv")
        negative = pd.read_csv("negative_tweet_" + suffix + ".csv")
        for column in positive.columns:
            if 'Unnamed' in column:
                positive.drop(column, axis=1, inplace=True)
        for column in negative.columns:
            if 'Unnamed' in column:
                negative.drop(column, axis=1, inplace=True)
        positives += [positive]
        negatives += [negative]
    return (positives, negatives)

def get_unique_number_of_tweets(arr_of_dataset):
    s = set()
    tot = 0
    for dataset in arr_of_dataset:
        s.update(set(dataset['id']))
        tot += len(dataset['id'])
    return (len(s), tot)

def merge_dataset(dataset,):
    ids = set(dataset[0]['id'])
    new_dataset = dataset[0].copy()
    for data in dataset[1:]:
        for idx, row in data.iterrows():
            if row['id'] in ids:
                continue
            new_dataset = new_dataset.append(row)
            ids.add(row['id'])
    new_dataset.sort_values('id', inplace=True)
    new_dataset.drop_duplicates(keep=False,inplace=True)
    return new_dataset

In [13]:
positives, negatives = read_pd_dataframe()
print('Positive (Unique, Total): ', get_unique_number_of_tweets(positives))
print('Negative (Unique, Total): ', get_unique_number_of_tweets(negatives))
positive_tweets, negative_tweets = merge_dataset(positives), merge_dataset(negatives)

print("Positive: ", len(positive_tweets))
print("Negative: ", len(negative_tweets))

Positive (Unique, Total):  (599, 1706)
Negative (Unique, Total):  (360, 1220)
Positive:  597
Negative:  358


In [15]:
len(set(positive_tweets['id'])) == len(positive_tweets)

True

In [16]:
dump_labeled_data('final.csv')

Positive:  597  Negative:  358  Total :  955
